# Experiment with transfer learning models b4 uploading to GCP

In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from glob import glob
import matplotlib.pyplot as plt
import sklearn.neighbors
from imblearn.under_sampling import RandomUnderSampler
import imageio
from PIL import Image

In [2]:
%load_ext autoreload
%autoreload 2

In [5]:
from skin_lesion_detection.data import get_data, clean_df, balance_nv, data_augmentation
from skin_lesion_detection.encoders import ImageScaler
# from skin_lesion_detection.trainer import Trainer
from skin_lesion_detection.transfer_learning_models import TLModels

## Get and Clean Data

In [6]:
df = get_data(50)

In [7]:
df.shape

(10015, 12)

In [8]:
df = clean_df(df)

/Users/AronSarosi/.pyenv/versions/3.7.7/envs/lesion_detection/lib/python3.7/site-packages/pandas/core/generic.py:5159: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
df.shape

(7470, 12)

In [10]:
df = balance_nv(df, 5)

In [11]:
df.shape

(2070, 12)

In [12]:
df = data_augmentation(df)

images_array.shape
        lesion_id      image_id     dx    dx_type   age     sex  \
0     HAM_0000897  ISIC_0029306     nv      histo  35.0  female   
1     HAM_0002088  ISIC_0024377     nv      histo  45.0  female   
2     HAM_0001568  ISIC_0030614     nv  follow_up  50.0    male   
3     HAM_0000118  ISIC_0027419    bkl      histo  80.0    male   
4     HAM_0002730  ISIC_0026769    bkl      histo  80.0    male   
...           ...           ...    ...        ...   ...     ...   
2065  HAM_0000193  ISIC_0030877  akiec      histo  60.0  female   
2066  HAM_0000020  ISIC_0031922  akiec      histo  60.0  female   
2067  HAM_0005579  ISIC_0029309  akiec      histo  80.0    male   
2068  HAM_0001576  ISIC_0033705  akiec      histo  60.0    male   
2069  HAM_0002867  ISIC_0033084  akiec      histo  40.0    male   

         localization                                               path  \
0               acral  ../dataset/HAM10000_images_part_2/ISIC_0029306...   
1     lower extremity  .

In [13]:
df.shape

(4140, 11)

In [14]:
df.tail(10)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx,images_resized
4130,HAM_0006024,ISIC_0029830,akiec,histo,60.0,male,face,../dataset/HAM10000_images_part_2/ISIC_0029830...,Actinic keratoses,0,"[174.8048, 144.8048, 173.0, 174.23602, 144.472..."
4131,HAM_0001036,ISIC_0027588,akiec,histo,50.0,female,face,../dataset/HAM10000_images_part_1/ISIC_0027588...,Actinic keratoses,0,"[86.0, 58.0, 52.0, 87.74881, 60.24847, 54.4983..."
4132,HAM_0004462,ISIC_0027334,akiec,histo,45.0,male,trunk,../dataset/HAM10000_images_part_1/ISIC_0027334...,Actinic keratoses,0,"[134.0, 93.0, 90.0, 134.0, 93.0, 90.0, 134.0, ..."
4133,HAM_0001152,ISIC_0030133,akiec,histo,65.0,male,face,../dataset/HAM10000_images_part_2/ISIC_0030133...,Actinic keratoses,0,"[218.70392, 163.18431, 128.18431, 218.31027, 1..."
4134,HAM_0004282,ISIC_0033811,akiec,histo,65.0,female,face,../dataset/HAM10000_images_part_2/ISIC_0033811...,Actinic keratoses,0,"[168.31349, 142.58043, 149.53485, 172.6102, 14..."
4135,HAM_0000193,ISIC_0030877,akiec,histo,60.0,female,face,../dataset/HAM10000_images_part_2/ISIC_0030877...,Actinic keratoses,0,"[216.37645, 190.25871, 215.50581, 217.25111, 1..."
4136,HAM_0000020,ISIC_0031922,akiec,histo,60.0,female,face,../dataset/HAM10000_images_part_2/ISIC_0031922...,Actinic keratoses,0,"[200.241, 167.11983, 171.13625, 202.187, 168.9..."
4137,HAM_0005579,ISIC_0029309,akiec,histo,80.0,male,face,../dataset/HAM10000_images_part_2/ISIC_0029309...,Actinic keratoses,0,"[146.44826, 119.03063, 135.59769, 148.11661, 1..."
4138,HAM_0001576,ISIC_0033705,akiec,histo,60.0,male,face,../dataset/HAM10000_images_part_2/ISIC_0033705...,Actinic keratoses,0,"[172.0, 131.07062, 133.07062, 172.76964, 129.9..."
4139,HAM_0002867,ISIC_0033084,akiec,histo,40.0,male,abdomen,../dataset/HAM10000_images_part_2/ISIC_0033084...,Actinic keratoses,0,"[159.21747, 113.362434, 121.39868, 160.44983, ..."


## Build CNN models